In [1]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from tqdm.autonotebook import tqdm
import ast
from lyricsgenius import Genius
import numpy as np
from spacy_langdetect import LanguageDetector
import spacy
import os
import translators as ts
import ast
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
import re
from matplotlib.colors import LogNorm
# communities
import community.community_louvain as cl
import csv
# coloring
from matplotlib import colors as mcolors
# graph
import networkx as nx
import matplotlib.pyplot as plt
from fa2 import ForceAtlas2
from scipy.stats import mode

C:\Users\malgo\AppData\Local\Temp\ipykernel_18276\1488235823.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm
Using state Capital Region server backend.


# GETTING DATA FROM SPOTIFY

First, we got 100 most streamed artist from [ChartMasters](https://chartmasters.org/most-streamed-artists-ever-on-spotify/).

In [2]:
client_id = "fe1069f5c28e4dc18d28cf23e64e6f61"
client_secret = "915fff01301f44c3b6faa8c12810bae6"

credmanager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=credmanager, requests_timeout=20, retries=10)

In [3]:
popular_artists = ["Drake", "Bad Bunny", "Ed Sheeran", "The Weekend", "Taylor Swift", "Justin Bieber", "Ariana Grande", 
                   "Eminem", "Post Malone", "BTS", "J Balvin", "Kanye West", "Billie Eilish", "Juice WRLD", "Coldplay", 
                   "Dua Lipa", "Imagine Dragons", "XXXTENTACION", "Ozuna", "Khalid", "Rihanna", "Maroon 5", "Travis Scott", 
                   "David Guetta", "Shawn Mendes", "Bruno Mars", "Daddy Yankee", "Calvin Harris", "Kendrick Lamar", "Sam Smith", 
                   "Queen", "Beyoncé", "Future", "Chris Brown", "One Direction", "The Chainsmokers", "Lady Gaga", "J. Cole", 
                   "Nicki Minaj", "Anuel AA", "Adele", "Halsey", "Selena Gomez", "The Beatles", "Harry Styles", "Sia", "Maluma", 
                   "Marshmello", "Lil Uzi Vert", "Linkin Park", "Twenty One Pilots", "Lana Del Rey", "Katy Perry", 
                   "Rauw Alejandro", "Kygo", "Avicii", "Doja Cat", "Farruko", "Camila Cabello", "Arctic Monkeys", "KAROL G", 
                   "Jason Derulo", "Red Hot Chili Peppers", "Shakira", "Demi Lovato", "OneRepublic", "Miley Cyrus", "Nicky Jam", 
                   "Michael Jackson", "Myke Towers", "Charlie Puth", "21 Savage", "Lil Baby", "Pitbull", "Mac Miller", "DaBaby", 
                   "Martin Garrix", "Pop Smoke", "Sebastian Yatra", "Olivia Rodrigo", "Cardi B", "Panic! At The Disco", 
                   "Metallica", "Sech", "Lil Wayne", "Lil Nas X", "Young Thug", "Ellie Goulding", "G-Eazy", "Alan Walker", 
                   "Lil Peep", "Major Lazer", "Wiz Khalifa", "Elton John", "Fall Out Boy", "AC/DC", "P!nk", "Tyga", 
                   "Trippie Redd", "John Mayer"]

### Relations between artists

Using Spotify API, we got 10 related artists for every popular artist. Then, we also got 10 related artists for every artist related to popular artists. This resulted in having a total of 2206 artists together with their correponding id and a list of relations between artists.

In [100]:
set_all_artists = set(popular_artists) # all artists that we have, those who have relations and those who are only related to someone else
list_of_related_artists = []
artists_id = {}
artists_already_having_relations = set() # if artist already exist in our list and has relations, we don't want to add him the second time
for artist in tqdm(popular_artists, total=len(popular_artists)): # first get only 
    artist_features = sp.search(q='artist:' + artist, type="artist")
    artist_id = artist_features["artists"]["items"][0]["id"]
    artist_name = artist_features["artists"]["items"][0]["name"]
    artists_id[artist_id] = artist_name
    
    related_artists = sp.artist_related_artists(artist_id)
    temp_list_of_related_artists = [] # list of artists related to the most popular artists
    
    for ra in related_artists["artists"][:10]:
        ra_name = ra["name"]
        ra_id = ra["id"]
        artists_id[ra_id] = ra_name
        
        temp_list_of_related_artists.append(ra_id) # list of artists related to the artists related to the most popular artists
        double_related_artists = sp.artist_related_artists(ra_id)
        temp_list_of_double_related_artists = []
        for double_ra in double_related_artists["artists"][:10]:
            double_ra_name = double_ra["name"]
            double_ra_id = double_ra["id"]
            artists_id[double_ra_id] = double_ra_name
            
            temp_list_of_double_related_artists.append(double_ra_id) 
            
        if ra_id not in artists_already_having_relations: # check if they don't already exist in our list
            list_of_related_artists.append({"id": ra_id,  "related_artists": temp_list_of_double_related_artists})
            artists_already_having_relations.add(ra_id)
        set_all_artists = set_all_artists | set(temp_list_of_double_related_artists)
        
    if artist_id not in artists_already_having_relations: # check if they don't already exist in our list
        list_of_related_artists.append({"id": artist_id, "related_artists": temp_list_of_related_artists})
        artists_already_having_relations.add(artist_id)
    set_all_artists = set_all_artists | set(temp_list_of_related_artists)

  0%|          | 0/100 [00:00<?, ?it/s]

In [108]:
artists_relations_df = pd.DataFrame(list_of_related_artists)
artists_relations_df.head()

,id,related_artists
0,0c173mlxpT3dSFRgMO8XPh,"[67nwj3Y5sZQLl72VNUHEYE, 17lzZA2AlOHwCwFALHttm..."
1,6l3HvQ5sa6mXTsMTB19rO5,"[0c173mlxpT3dSFRgMO8XPh, 67nwj3Y5sZQLl72VNUHEY..."
2,20sxb77xiYeusSH8cVdatc,"[1sBkRIssrMs1AbVkOJbc7a, 6vXTefBL93Dj5IqAWq6OT..."
3,0QHgL1lAIqAw0HtD7YldmP,"[1sBkRIssrMs1AbVkOJbc7a, 5OrB6Jhhrl9y2PK0pSV4V..."
4,2HPaUgqeutzr3jx5a9WyDV,"[4HzKw8XcD0piJmDrrPRCYk, 7mDU6nMUJnOSY2Hkjz5oq..."


In [154]:
artists_relations_df.to_csv("artists_relations.csv", sep=";", index=None) # file with relations between artists

In [116]:
artists_id_df = pd.DataFrame(artists_id.items(), columns=["id", "name"])
artists_id_df.head()

,id,name
0,3TVXtAsR1Inumwj472S9r4,Drake
1,0c173mlxpT3dSFRgMO8XPh,Big Sean
2,67nwj3Y5sZQLl72VNUHEYE,Wale
3,17lzZA2AlOHwCwFALHttmp,2 Chainz
4,1sBkRIssrMs1AbVkOJbc7a,Rick Ross


In [152]:
artists_id_df.to_csv("artists_id.csv", index=None) # artists names and ids

### Data about artists and tracks

Once data about relations between artists was ready, more specific data about each artist, such as **number of followers**, **genres** and **popularity** were downloaded. What's more, for every artist top 10 tracks were downloaded, together with its features: **album id**, **album name**, **album release data**, **number of tracks on album**, **artists collaborating in the track**, **track's popularity**, **track id**, **number of track on album**, **danceability**, **energy**, **key**, **loudness**, **mode**, **speechiness**, **acousticness**, **instrumentalness**, **liveness**, **valence** and **tempo**. For some artists there were less than 10 top tracks possible, so finally it resulted in total of 21985 tracks. Also, not all features were available for all tracks, so there are some missing data. Tracks were saved as a list of dictionaries for every artist.

In [3]:
artists_id_df = pd.read_csv("artists_id.csv")
artists_id_df.head()

,id,name
0,3TVXtAsR1Inumwj472S9r4,Drake
1,0c173mlxpT3dSFRgMO8XPh,Big Sean
2,67nwj3Y5sZQLl72VNUHEYE,Wale
3,17lzZA2AlOHwCwFALHttmp,2 Chainz
4,1sBkRIssrMs1AbVkOJbc7a,Rick Ross


In [4]:
artists_data_list = []
for id_ in tqdm(artists_id_df["id"], total=len(artists_id_df)):
    
    results = sp.artist(id_) # get basic data about artist
    data = {"id": id_, "name": results["name"], "number_of_followers": results["followers"]["total"], 
            "genres": results["genres"], "popularity": results["popularity"]}
    top_tracks_results = sp.artist_top_tracks(id_) # get basic data about top tracks
    tracks_data = []
    for track in top_tracks_results["tracks"]:
        temp_track_data = {}
        temp_audio_data = {}
        
        all_track_artists = [] # information about all artists in the track
        for a in track["artists"]:
            all_track_artists.append(a["name"])
            
        temp_track_data = {"album_id": track.get("album", {}).get("id"), "album_name": track.get("album", {}).get("name"), 
                            "album_release_data": track.get("album", {}).get("release_date"), 
                            "number_of_tracks_on_album": track.get("album", {}).get("total_tracks"), 
                            "artists": all_track_artists, 
                            "id": track.get("id"), "name": track.get("name"), "popularity": track.get("popularity"), 
                            "track_number_on_album": track.get("track_number")}
            
        audio_data = sp.audio_features(track["id"])[0] # get specific track data (they don't always exist)
        if audio_data:
            temp_audio_data = {"danceability": audio_data.get("danceability"), 
                               "energy": audio_data.get("energy"), "key": audio_data.get("key"), 
                               "loudness": audio_data.get("loudness"), "mode": audio_data.get("mode"), 
                               "speechiness": audio_data.get("speechiness"), "acousticness": audio_data.get("acousticness"), 
                               "instrumentalness": audio_data.get("instrumentalness"), "liveness": audio_data.get("liveness"), 
                               "valence": audio_data.get("valence"), "tempo": audio_data.get("tempo")}
            temp_track_data.update(temp_audio_data)
            
        tracks_data.append(temp_track_data)
    data["top_tracks"] = tracks_data
    artists_data_list.append(data)

  0%|          | 0/2206 [00:00<?, ?it/s]

In [5]:
artists_data_df = pd.DataFrame(artists_data_list)
artists_data_df.head()

,id,name,number_of_followers,genres,popularity,top_tracks
0,3TVXtAsR1Inumwj472S9r4,Drake,68477309,"[canadian hip hop, canadian pop, hip hop, rap,...",96,"[{'album_id': '3cf4iSSKd8ffTncbtKljXw', 'album..."
1,0c173mlxpT3dSFRgMO8XPh,Big Sean,10154240,"[detroit hip hop, hip hop, pop, pop rap, r&b, ...",77,"[{'album_id': '2HaqChIDc5go3qxVunBDK0', 'album..."
2,67nwj3Y5sZQLl72VNUHEYE,Wale,3836361,"[gangster rap, hip hop, pop rap, r&b, rap, sou...",69,"[{'album_id': '0jCVC8ndYYOooEY2YTO1l6', 'album..."
3,17lzZA2AlOHwCwFALHttmp,2 Chainz,7462599,"[atl hip hop, gangster rap, hip hop, pop rap, ...",75,"[{'album_id': '35KyVKa5AMTaOBxSzjSBbK', 'album..."
4,1sBkRIssrMs1AbVkOJbc7a,Rick Ross,6359211,"[dirty south rap, gangster rap, hip hop, pop r...",76,"[{'album_id': '00HZxSA1sOS9T3nA950sg4', 'album..."


In [12]:
artists_data_df.to_csv("artists_data.csv", sep=";", index=False)

### Additional file with tracks

To enable more clear and transparent coding and analysis, a new file with tracks was created. At first there was one row for every artist, and for every artist there was a list of dictionaries with tracks in column top_tracks. In a new file every row is a separate track, and features of tracks are in separate columns.

In [3]:
artists_data_df = pd.read_csv("artists_data.csv", sep=";")
artists_data_df.head()

,id,name,number_of_followers,genres,popularity,top_tracks
0,3TVXtAsR1Inumwj472S9r4,Drake,68477309,"['canadian hip hop', 'canadian pop', 'hip hop'...",96,"[{'album_id': '3cf4iSSKd8ffTncbtKljXw', 'album..."
1,0c173mlxpT3dSFRgMO8XPh,Big Sean,10154240,"['detroit hip hop', 'hip hop', 'pop', 'pop rap...",77,"[{'album_id': '2HaqChIDc5go3qxVunBDK0', 'album..."
2,67nwj3Y5sZQLl72VNUHEYE,Wale,3836361,"['gangster rap', 'hip hop', 'pop rap', 'r&b', ...",69,"[{'album_id': '0jCVC8ndYYOooEY2YTO1l6', 'album..."
3,17lzZA2AlOHwCwFALHttmp,2 Chainz,7462599,"['atl hip hop', 'gangster rap', 'hip hop', 'po...",75,"[{'album_id': '35KyVKa5AMTaOBxSzjSBbK', 'album..."
4,1sBkRIssrMs1AbVkOJbc7a,Rick Ross,6359211,"['dirty south rap', 'gangster rap', 'hip hop',...",76,"[{'album_id': '00HZxSA1sOS9T3nA950sg4', 'album..."


In [17]:
all_track_data = []
for i, row in tqdm(artists_data_df.iterrows(), total = len(artists_data_df)):
    artist_id = row["id"]
    artist_name = row["name"]
    artist_no_of_followers = row["number_of_followers"]
    artist_genres = row["genres"]
    artist_popularity = row["popularity"]
    for track in ast.literal_eval(row["top_tracks"]):
        track_data = {"artist_id": artist_id, "artist_name": artist_name, "number_of_followers": artist_no_of_followers,
                      "artist_genres": artist_genres, "artist_popularity": artist_popularity}
        track_data.update({"album_id": track.get("album_id"), "album_name": track.get("album_name"), 
                           "album_release_data": track.get("album_release_data"), 
                           "number_of_tracks_on_album": track.get("number_of_tracks_on_album"), 
                           "track_artists": track.get("artists"), "track_id": track.get("id"), "track_name": track.get("name"), 
                           "track_popularity": track.get("popularity"), "track_number_on_album": track.get("track_number_on_album"), 
                           "danceability": track.get("danceability"), "energy": track.get("energy"), "key": track.get("key"), 
                           "loudness": track.get("loudness"), "mode": track.get("mode"), "speechiness": track.get("speechiness"), 
                           "acousticness": track.get("acousticness"), "instrumentalness": track.get("instrumentalness"), 
                           "liveness": track.get("liveness"), "valence": track.get("valence"), "tempo": track.get("tempo")})
        all_track_data.append(track_data)

  0%|          | 0/2206 [00:00<?, ?it/s]

In [18]:
pd.set_option("display.max_columns", None)
track_df = pd.DataFrame(all_track_data)
track_df.head()

,artist_id,artist_name,number_of_followers,artist_genres,artist_popularity,album_id,album_name,album_release_data,number_of_tracks_on_album,track_artists,track_id,track_name,track_popularity,track_number_on_album,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,3TVXtAsR1Inumwj472S9r4,Drake,68477309,"['canadian hip hop', 'canadian pop', 'hip hop'...",96,3cf4iSSKd8ffTncbtKljXw,"Honestly, Nevermind",2022-06-17,14,"[Drake, 21 Savage]",3F5CgOj3wFlRv51JsHbxhe,Jimmy Cooks (feat. 21 Savage),90,14,0.529,0.673,0.0,-4.711,1.0,0.175,0.000307,0.000002,0.0930,0.366,165.921
1,3TVXtAsR1Inumwj472S9r4,Drake,68477309,"['canadian hip hop', 'canadian pop', 'hip hop'...",96,5MS3MvWHJ3lOZPLiMxzOU6,Her Loss,2022-11-04,16,"[Drake, 21 Savage]",1bDbXMyjaUIooNwFE9wn0N,Rich Flex,89,1,0.561,0.520,11.0,-9.342,0.0,0.244,0.050300,0.000002,0.3550,0.424,153.150
2,3TVXtAsR1Inumwj472S9r4,Drake,68477309,"['canadian hip hop', 'canadian pop', 'hip hop'...",96,6tE9Dnp2zInFij4jKssysL,I NEVER LIKED YOU,2022-04-29,16,"[Future, Drake, Tems]",59nOXPmaKlBfGMDeOVGrIK,WAIT FOR U (feat. Drake & Tems),88,7,0.463,0.642,1.0,-4.474,1.0,0.340,0.314000,0.000000,0.0686,0.339,83.389
3,3TVXtAsR1Inumwj472S9r4,Drake,68477309,"['canadian hip hop', 'canadian pop', 'hip hop'...",96,5MS3MvWHJ3lOZPLiMxzOU6,Her Loss,2022-11-04,16,"[Drake, 21 Savage]",46s57QULU02Voy0Kup6UEb,Major Distribution,88,2,0.908,0.546,8.0,-10.491,0.0,0.323,0.012700,0.000000,0.0707,0.225,131.016
4,3TVXtAsR1Inumwj472S9r4,Drake,68477309,"['canadian hip hop', 'canadian pop', 'hip hop'...",96,5MS3MvWHJ3lOZPLiMxzOU6,Her Loss,2022-11-04,16,"[Drake, 21 Savage, Travis Scott]",2KLwPaRDOB87XOYAT2fgxh,Pussy & Millions (feat. Travis Scott),87,10,0.749,0.630,4.0,-6.652,0.0,0.116,0.062300,0.000000,0.3530,0.445,122.030


In [20]:
track_df.to_csv("track_data.csv", sep=";", index=False)

# GETTING LYRICS

Track lyrics were downloaded for text analysis using Genius API. Unfortunately, it was impossible to get lyrics for all songs, so at the end there were 18451 files with lyrics out of 21985 tracks. What's more, not all files were correct. Some of them, instead of actual lyrics, contained essays, poems, chapters from books etc. Many of them were removed manually which resulted in 17649 files.

In [ ]:
tracks = pd.read_csv("track_data.csv", sep=";")
index_list = tracks.track_name.keys().to_list()

In [ ]:
genius = Genius(access_token = "oiDi4isGxovUcT1wDOGjqxBkDZs-eSSRSi6r3KEL9Gs-6VkArNdnoiXwrHAupoJl", verbose = False,
               remove_section_headers = True, skip_non_songs = True)

In [ ]:
notFound = []
trackList = tracks.track_name.tolist()
for i in tqdm(index_list):
    try:
        song = genius.search_song(tracks.iloc[i].track_name)
        Lyrics = song.lyrics
            
        #save lyrics
        text_file = open("Lyrics/" + str(i) + ".txt", "w", encoding = 'utf-8')
        text_file.write(Lyrics)
        text_file.close()
    except:
        print("exception at:", i)
        notFound.append(i)

# LYRICS TRANSLATION

Some of the songs weren't in English. Why we decided to automatically translate those lyrics to ensure proper text analysis. Of course, one may argue that automatic translations may not be accurate enough, but since today's translators can deal not only with short, easy sentences, but also with longer, complicated documents or slang, the results should be good enough to use in text analysis. And indeed, after manually checking translated texts we decided to use them in further analysis.

The process of automatic detection was as follows: first, language of text was detected using language detector from spaCy module. If lyrics weren't in English, they were translated using google translator implemented in module translators.

We need to download spacy models in order to make the code below work:
* python -m spacy download en_core_web_sm

In [ ]:
def get_lang_detector(nlp, name):
    return LanguageDetector()

In [ ]:
nlp = spacy.load("en_core_web_sm")  # initiating language detector
spacy.Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)

In [ ]:
lyrics_directory = "Lyrics"
translated_lyrics_directory = "Lyrics_english" # directory to store English version of lyrics
files = os.listdir(lyrics_directory)
os.makedirs(translated_lyrics_directory, exist_ok=True) 
files_translated = os.listdir(translated_lyrics_directory)

for file in tqdm(files, total=len(files)):
    if ".txt" in file and file not in files_translated:
        with open(lyrics_directory + "/" + file, "r", encoding="utf-8") as f:
            lyrics_text = f.read()
        doc = nlp(lyrics_text) 
        detect_language = doc._.language  # detecting lyrics language
        if detect_language["language"] != "en": # translating if lyrics not in English
            if len(lyrics_text) >= 4000: # google translator can't translate texts with more than 4000 characters
                split_lyrics = lyrics_text.split("\n") # splitting text in lines if it's too long and translating separate lines
                split_lyrics_translated = []
                for line in tqdm(split_lyrics, total=len(split_lyrics)):
                    split_lyrics_translated.append(ts.google(line, from_language=detect_language["language"], to_language='en'))
                lyrics_english = "\n".join(split_lyrics_translated)
            else:
                lyrics_english = ts.google(lyrics_text, from_language=detect_language["language"], to_language='en')
        else:
            lyrics_english = lyrics_text
        with open(translated_lyrics_directory + "/" + file, "w", encoding = "utf-8") as f:
            f.write(lyrics_english)